In [ ]:
GPU_PKG_NAME = "AMDGPU"; include("common_definitions.jl");

In [1]:
using OffsetArrays
using Images

┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1317


Heat diffusion with broadcast, why the heck not

Inspired by https://enccs.github.io/OpenACC-CUDA-beginners/2.02_cuda-heat-equation/

In [2]:
Base.Broadcast.BroadcastStyle(::Type{<:OffsetArray{<:Any, <:Any, AA}}) where AA = Base.Broadcast.BroadcastStyle(AA)

In [3]:
a  = 0.01
dx = 0.01  # x-grid spacing
dy = 0.01  # y-grid spacing

0.01

In [4]:
dt = dx^2 * dy^2 / (2.0 * a * (dx^2 + dy^2)) # Largest stable time step

0.0024999999999999996

In [5]:
N = 64

64

In [6]:
function diffuse!(data, a, dt, dx, dy)
    dij   = view(data, 1:N, 1:N)
    dim1j = view(data, 0:(N-1), 1:N)
    dijm1 = view(data, 1:N, 0:(N-1))
    dip1j = view(data, 2:(N+1), 1:N)
    dijp1 = view(data, 1:N, 2:(N+1))

    # Apply diffusion
    @. dij += a * dt * (
        (dim1j - 2 * dij + dip1j)/dx^2 +
        (dijm1 - 2 * dij + dijp1)/dy^2)

    # update boundary condition (wrap around)
    data[0, :]   .= data[N, :]
    data[N+1, :] .= data[1, :]
    data[:, 0]   .= data[:, N]
    data[:, N+1] .= data[:, 0]
    data
end

diffuse! (generic function with 1 method)

In [18]:
using Reel; Reel.set_output_type("gif")
frames = Frames(MIME("image/png"), fps=30)
domain = OffsetArray(zeros(N+2, N+2), 0:(N+1), 0:(N+1))
domain[16:32, 16:32] .= 5
for i in 1:1000
    diffuse!(domain, a, dt, dx, dy)
    push!(frames, Gray.(domain))
end
frames

Frames{MIME{Symbol("image/png")}}("/tmp/jl_8d2OhI", 0x00000000000003e8, 30.0, nothing)